# Reproducibility - resume

1. Create and commit two versions of dataframe -> v1, v2
2. Run flow 1 with v1 data -> crashes b/c flow 2 hasn't run
3. Run flow 2 with v2 data
4. Resume flow 1 successfully

In [3]:
import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init("foo")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

01-17 16:53:34 doltpy.core.dolt INFO     Creating directory foo
01-17 16:53:34 doltpy.core.dolt INFO     Creating a new repo in foo
01-17 16:53:34 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
01-17 16:53:34 doltpy.core.write.write INFO     Importing to table bar in dolt directory located in foo, import mode create
01-17 16:53:35 doltpy.core.dolt INFO     Rows Processed: 3, Additions: 3, Modifications: 0, Had No Effect: 0
01-17 16:53:35 doltpy.core.dolt INFO     
01-17 16:53:35 doltpy.core.dolt INFO     commit ldjnaiqi9r5n22lalefvt5r2e3eipjkt
Author: Max Hoffman <maximilian.wolfgang1@gmail.com>
Date:   Sun Jan 17 16:53:35 -0800 2021

	Initialize bar


01-17 16:53:35 doltpy.core.write.write INFO     Importing to table bar in dolt directory located in foo, import mode update
01-17 16:53:35 doltpy.core.dolt INFO     Rows Processed: 6, Additions: 3, Modifications: 0, Had No Effect: 3
01-17 16:53:35 doltpy.core.dolt INFO     
01-17 16:53:35

In [2]:
!cat succeeds_second.py

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter, Flow
import pandas as pd
from sklearn import tree

class SucceedsSecondDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            self.df = dolt.read_table('bar')
        first = Flow("SucceedsFirstDemo").latest_successful_run

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    SucceedsSecondDemo()


In [36]:
!cat succeeds_first.py

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter
import pandas as pd
from sklearn import tree

class SucceedsFirstDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, doltdb_path='foo', branch=self.bar_version) as dolt:
            self.df = dolt.read_table('bar')

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, doltdb_path='foo', branch=self.bar_version) as dolt:

            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    SucceedsFirstDemo()


In [20]:
!poetry run python3 succeeds_second.py run --bar-version $v1

Metaflow 2.2.5.post22+git5ab5047 executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 17:37:45.407 Workflow starting (run-id 1610933865398710):
2021-01-17 17:37:45.418 [1610933865398710/start/1 (pid 29031)] Task is starting.
2021-01-17 17:37:46.482 [1610933865398710/start/1 (pid 29031)] 01-17 17:37:46 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 17:37:46.536 [1610933865398710/start/1 (pid 29031)] 01-17 17:37:46 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 17:37:46.804 [1610933865398710/start/1 (pid 29031)] 01-17 17:37:46 doltpy.core.dolt INFO     * master                                        	p6l77ib7o7gglu0eg6s748il078kj4b5
2021-01-17 17:37:46.973 [1610933865398710/start/1 (pid 29031)] 
2021-01-17 17:37:46.973 [1610933865398710/start/1 (pid 29031)] 01-17 17:37:46 

01-17 17:37:47 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [21]:
!poetry run python3 succeeds_first.py run --bar-version $v2

Metaflow 2.2.5.post22+git5ab5047 executing SucceedsFirstDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 17:38:05.472 Workflow starting (run-id 1610933885461124):
2021-01-17 17:38:05.480 [1610933885461124/start/1 (pid 29078)] Task is starting.
2021-01-17 17:38:06.613 [1610933885461124/start/1 (pid 29078)] 01-17 17:38:06 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 17:38:06.642 [1610933885461124/start/1 (pid 29078)] 01-17 17:38:06 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 17:38:06.703 [1610933885461124/start/1 (pid 29078)] 01-17 17:38:06 doltpy.core.dolt INFO     * master                                        	p6l77ib7o7gglu0eg6s748il078kj4b5
2021-01-17 17:38:06.875 [1610933885461124/start/1 (pid 29078)] 
2021-01-17 17:38:06.875 [1610933885461124/start/1 (pid 29078)] 01-17 17:38:06 d

2021-01-17 17:38:09.642 [1610933885461124/middle/2 (pid 29112)] Task finished successfully.
2021-01-17 17:38:09.649 [1610933885461124/end/3 (pid 29152)] Task is starting.
2021-01-17 17:38:10.822 [1610933885461124/end/3 (pid 29152)] 01-17 17:38:10 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 17:38:10.829 [1610933885461124/end/3 (pid 29152)] 01-17 17:38:10 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 17:38:10.987 [1610933885461124/end/3 (pid 29152)] Task finished successfully.
2021-01-17 17:38:10.988 Done!
01-17 17:38:10 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 17:38:10 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [34]:
!poetry run python3 succeeds_second.py resume start

Metaflow 2.2.5.post19+gitd26b8cd executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-15 15:38:34.003 Gathering required information to resume run (this may take a bit of time)...
2021-01-15 15:38:34.011 Workflow starting (run-id 1610753914002543):
2021-01-15 15:38:34.018 [1610753914002543/start/1 (pid 12116)] Task is starting.
2021-01-15 15:38:35.150 [1610753914002543/start/1 (pid 12116)] 01-15 15:38:35 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-15 15:38:35.240 [1610753914002543/start/1 (pid 12116)] 01-15 15:38:35 doltpy.core.dolt INFO       master                                        	r44hf3g77op6cfhmhj10fpa59qu94p00
2021-01-15 15:38:35.365 [1610753914002543/start/1 (pid 12116)] * tmp_4823254690243868365                       	0cbd00l60kvopvd3qmcohm8ej44pdp2b
2021-01-15 15:38:35.365 [1610753914002543/start/1 (pid 12116)]   tmp_5

2021-01-15 15:38:39.040 [1610753914002543/end/3 (pid 12181)] 01-15 15:38:39 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-15 15:38:39.202 [1610753914002543/end/3 (pid 12181)] Task finished successfully.
2021-01-15 15:38:39.203 Done!
01-15 15:38:39 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-15 15:38:39 doltpy.core.system_helpers INFO     No processes to clean up, exiting
